# 01 — Price Index Prototype

Pull MLIT transaction data for Tokyo’s 23 wards and Sendai’s wards.

## 1. Setup

In [4]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import re

# Load MLIT API key from ../.env
MLIT_API_KEY = None
for line in Path('../.env').read_text().splitlines():
    if line.startswith('MLIT_API_KEY'):
        MLIT_API_KEY = line.split('=', 1)[1].strip().strip('"')
        break

assert MLIT_API_KEY, 'MLIT_API_KEY not found in ../.env'
HEADERS = {'Ocp-Apim-Subscription-Key': MLIT_API_KEY}

# Target time window (adjust as needed)
YEARS = list(range(2023, 2024))
QUARTERS = [1, 2, 3, 4]


## 2. Municipality Lists

In [5]:
TOKYO_WARDS = [
    {'id': '13101', 'name': 'Chiyoda Ward'},
    {'id': '13102', 'name': 'Chuo Ward'},
    {'id': '13103', 'name': 'Minato Ward'},
    {'id': '13104', 'name': 'Shinjuku Ward'},
    {'id': '13105', 'name': 'Bunkyo Ward'},
    {'id': '13106', 'name': 'Taito Ward'},
    {'id': '13107', 'name': 'Sumida Ward'},
    {'id': '13108', 'name': 'Koto Ward'},
    {'id': '13109', 'name': 'Shinagawa Ward'},
    {'id': '13110', 'name': 'Meguro Ward'},
    {'id': '13111', 'name': 'Ota Ward'},
    {'id': '13112', 'name': 'Setagaya Ward'},
    {'id': '13113', 'name': 'Shibuya Ward'},
    {'id': '13114', 'name': 'Nakano Ward'},
    {'id': '13115', 'name': 'Suginami Ward'},
    {'id': '13116', 'name': 'Toshima Ward'},
    {'id': '13117', 'name': 'Kita Ward'},
    {'id': '13118', 'name': 'Arakawa Ward'},
    {'id': '13119', 'name': 'Itabashi Ward'},
    {'id': '13120', 'name': 'Nerima Ward'},
    {'id': '13121', 'name': 'Adachi Ward'},
    {'id': '13122', 'name': 'Katsushika Ward'},
    {'id': '13123', 'name': 'Edogawa Ward'},
]

SENDAI_WARDS = [
    {'id': '04101', 'name': 'Aoba Ward'},
    {'id': '04102', 'name': 'Miyagino Ward'},
    {'id': '04103', 'name': 'Wakabayashi Ward'},
    {'id': '04104', 'name': 'Taihaku Ward'},
    {'id': '04105', 'name': 'Izumi Ward'},
]

pd.DataFrame(TOKYO_WARDS + SENDAI_WARDS).head()


,id,name
0,13101,Chiyoda Ward
1,13102,Chuo Ward
2,13103,Minato Ward
3,13104,Shinjuku Ward
4,13105,Bunkyo Ward


## 3. Pull Transactions (one request loop)

In [6]:
def fetch_transactions(city_code, year, quarter, language='en'):
    response = requests.get(
        'https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001',
        headers=HEADERS,
        params={
            'priceClassification': '01',
            'year': year,
            'quarter': quarter,
            'city': city_code,
            'language': language,
        },
        timeout=60
    )
    response.raise_for_status()
    return response.json().get('data', [])

records = []
municipalities = TOKYO_WARDS + SENDAI_WARDS
for muni in municipalities:
    for year in YEARS:
        for quarter in QUARTERS:
            data = fetch_transactions(muni['id'], year, quarter)
            for row in data:
                row['CityCode'] = muni['id']
                row['AreaLabel'] = muni['name']
                row['YearParam'] = year
                row['QuarterParam'] = quarter
                records.append(row)
print(f'Transactions pulled: {len(records):,}')
transactions_df = pd.DataFrame(records)
transactions_df.head()


Transactions pulled: 26,454


,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,...,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Renovation,Remarks,CityCode,AreaLabel,YearParam,QuarterParam
0,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kandasakumacho,64000000,,1LDK,...,Commercial Zone,80,600,1st quarter 2023,Not yet,,13101,Chiyoda Ward,2023,1
1,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kandasakumacho,60000000,,1LDK,...,Commercial Zone,80,600,1st quarter 2023,Not yet,,13101,Chiyoda Ward,2023,1
2,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kandasakumacho,36000000,,1R,...,Commercial Zone,80,600,1st quarter 2023,Not yet,,13101,Chiyoda Ward,2023,1
3,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kandasakumacho,37000000,,1K,...,Commercial Zone,80,600,1st quarter 2023,Not yet,,13101,Chiyoda Ward,2023,1
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kandajimbocho,110000000,,1LDK,...,Commercial Zone,80,600,1st quarter 2023,Done,,13101,Chiyoda Ward,2023,1


In [7]:
transactions_df.to_csv("transactions.csv", index=False)

In [8]:
# if using csv directly
transactions_df = pd.read_csv("transactions.csv")
transactions_df.head()

,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,...,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Renovation,Remarks,CityCode,AreaLabel,YearParam,QuarterParam
0,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kandasakumacho,64000000,NaN,1LDK,...,Commercial Zone,80.0,600.0,1st quarter 2023,Not yet,NaN,13101,Chiyoda Ward,2023,1
1,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kandasakumacho,60000000,NaN,1LDK,...,Commercial Zone,80.0,600.0,1st quarter 2023,Not yet,NaN,13101,Chiyoda Ward,2023,1
2,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kandasakumacho,36000000,NaN,1R,...,Commercial Zone,80.0,600.0,1st quarter 2023,Not yet,NaN,13101,Chiyoda Ward,2023,1
3,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kandasakumacho,37000000,NaN,1K,...,Commercial Zone,80.0,600.0,1st quarter 2023,Not yet,NaN,13101,Chiyoda Ward,2023,1
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kandajimbocho,110000000,NaN,1LDK,...,Commercial Zone,80.0,600.0,1st quarter 2023,Done,NaN,13101,Chiyoda Ward,2023,1


In [10]:
transactions_df.columns

Index(['PriceCategory', 'Type', 'Region', 'MunicipalityCode', 'Prefecture',
       'Municipality', 'DistrictName', 'TradePrice', 'PricePerUnit',
       'FloorPlan', 'Area', 'UnitPrice', 'LandShape', 'Frontage',
       'TotalFloorArea', 'BuildingYear', 'Structure', 'Use', 'Purpose',
       'Direction', 'Classification', 'Breadth', 'CityPlanning',
       'CoverageRatio', 'FloorAreaRatio', 'Period', 'Renovation', 'Remarks',
       'CityCode', 'AreaLabel', 'YearParam', 'QuarterParam'],
      dtype='object')